In [ ]:
%pip install torch torchvision timm scikit-learn matplotlib seaborn h5py pillow


In [3]:
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
# from efficientnet_pytorch import EfficientNet
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader, random_split
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from PIL import Image

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Constants
IMG_SIZE = 300  # EfficientNetB3 input size
BATCH_SIZE = 32
NUM_EPOCHS = 30
LEARNING_RATE = 0.0001
MODEL_PATH = r'C:\Users\ssgou\Downloads\MPS\Mini Project\eye_disease_model.pth'

# Define classes
classes = ['cataract', 'DR', 'glaucoma', 'normal']
NUM_CLASSES = len(classes)


In [3]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # adjust for RGB images
])

# Load dataset
dataset = datasets.ImageFolder(root=r'C:\Users\rohit\Downloads\dataset', transform=transform)  # 🔁 Replace with actual dataset path

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)


In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=NUM_CLASSES)
model.to(DEVICE)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print("Using Adam optimizer.")


In [ ]:
train_losses = []
train_accuracies = []
label= []
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0

    model.train()
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

torch.save({
    'model_state_dict': model.state_dict(),
    'classes': classes,
    'input_size': IMG_SIZE
}, MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")


In [ ]:
from efficientnet_pytorch import EfficientNet
import torch
import torch.nn as nn
classes = ['cataract', 'DR', 'glaucoma', 'normal']
NUM_CLASSES = len(classes)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNet.from_name('efficientnet-b3')
model._fc = nn.Linear(model._fc.in_features, NUM_CLASSES)
MODEL_PATH = r'C:\Users\rohit\Downloads\Mini Project\eye_disease_model.pth'
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

print("Model loaded successfully for evaluation.")


In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

all_preds = []
all_labels = []
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes))

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



# plt.figure(figsize=(10, 4))
# plt.subplot(1, 2, 1)
# plt.plot(train_losses, label='Loss')
# plt.title('Training Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(train_accuracies, label='Accuracy')
# plt.title('Training Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()

# plt.tight_layout()
# plt.show()



In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
inference_model = EfficientNet.from_name('efficientnet-b3')
inference_model._fc = nn.Linear(inference_model._fc.in_features, NUM_CLASSES)

checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
inference_model.load_state_dict(checkpoint['model_state_dict'])
inference_model.to(DEVICE)
inference_model.eval()

print("Inference model loaded successfully.")


In [ ]:
def predict_image(image_path, model, transform, classes):
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    plt.imshow(image)
    plt.axis('off')
    plt.title("Input Image")
    plt.show()

    image_tensor = transform(image).unsqueeze(0).to(DEVICE)  # Add batch dimension

    # Predict
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        predicted_class = classes[predicted.item()]

    print(f"Predicted Class: {predicted_class}")

# 🔁 Example usage:
image_path = r"C:\Users\rohit\Downloads\Mini Project\split_dataset\test\glaucoma\_2_8820241.jpg"  # Replace with actual image path
predict_image(image_path, inference_model, transform, classes)
